# cjm-fasthtml-workers

> Background worker system for FastHTML with multiprocess job execution, cancellation support, and streaming capabilities.

## Install

```bash
pip install cjm_fasthtml_workers
```

## Project Structure

```
nbs/
├── core/ (4)
│   ├── adapters.ipynb  # Adapter utilities for making plugin managers compatible with the worker system.
│   ├── config.ipynb    # Configuration for worker processes including restart policies, timeouts, and queue sizes.
│   ├── protocol.ipynb  # Protocol definitions for worker communication and plugin manager integration.
│   └── worker.ipynb    # Generic worker process for executing plugin-based jobs in isolated subprocesses.
├── extensions/ (1)
│   └── protocols.ipynb  # Optional integration protocols for plugin registries, resource management, and event broadcasting.
└── managers/ (1)
    └── base.ipynb  # Abstract base class for managing background jobs with worker processes.
```

Total: 6 notebooks across 3 directories

## Module Dependencies

```mermaid
graph LR
    core_adapters[core.adapters<br/>adapters]
    core_config[core.config<br/>config]
    core_protocol[core.protocol<br/>protocol]
    core_worker[core.worker<br/>worker]
    extensions_protocols[extensions.protocols<br/>protocols]
    managers_base[managers.base<br/>base]

    core_adapters --> core_protocol
    core_worker --> core_protocol
    managers_base --> core_protocol
    managers_base --> core_config
    managers_base --> extensions_protocols
```

*5 cross-module dependencies detected*

## CLI Reference

No CLI commands found in this project.

## Module Overview

Detailed documentation for each module in the project:

### adapters (`adapters.ipynb`)
> Adapter utilities for making plugin managers compatible with the worker system.

#### Import

```python
from cjm_fasthtml_workers.core.adapters import (
    create_simple_adapter,
    default_result_adapter
)
```

#### Functions

```python
def create_simple_adapter(
    plugin_manager,  # The plugin manager instance to adapt
    result_adapter: Optional[callable] = None  # Optional function to convert plugin results to dict
) -> PluginManagerAdapter:  # Adapter that satisfies PluginManagerAdapter protocol
    """
    Create a simple adapter for a plugin manager.
    
    This is a convenience function that wraps a plugin manager object
    to satisfy the PluginManagerAdapter protocol.
    
    Example:
        >>> from some_plugin_system import PluginManager
        >>> pm = PluginManager()
        >>> adapter = create_simple_adapter(pm)
        >>> # adapter now satisfies PluginManagerAdapter protocol
    """
```

```python
def default_result_adapter(
    result: Any  # Plugin execution result
) -> Dict[str, Any]:  # Dictionary with text and metadata
    """
    Default adapter for converting plugin results to dictionaries.
    
    Assumes result has 'text' and 'metadata' attributes.
    """
```


### base (`base.ipynb`)
> Abstract base class for managing background jobs with worker processes.

#### Import

```python
from cjm_fasthtml_workers.managers.base import (
    JobType,
    BaseJob,
    BaseJobManager
)
```

#### Functions

```python
@patch
@abstractmethod
def create_job(
    self: BaseJobManager,
    plugin_id: str,  # Plugin unique identifier
    **kwargs  # Domain-specific job parameters
) -> JobType:  # Created job instance
    """
    Factory method for creating domain-specific jobs.
    
    Subclasses must implement this to create their specific job type.
    """
```

```python
@patch
@abstractmethod
def get_worker_entry_point(
    self: BaseJobManager
) -> Callable:  # Worker process entry point function
    """
    Return the worker process function for this manager.
    
    Subclasses must implement this to provide their worker entry point.
    """
```

```python
@patch
@abstractmethod
def prepare_execute_request(
    self: BaseJobManager,
    job: JobType  # The job to prepare for execution
) -> Dict[str, Any]:  # Dictionary of parameters for the worker execute request
    """
    Convert job to worker execute request parameters.
    
    Subclasses must implement this to extract execution parameters from their job type.
    """
```

```python
@patch
@abstractmethod
def extract_job_result(
    self: BaseJobManager,
    job: JobType,  # The job that was executed
    result_data: Dict[str, Any]  # Raw result data from worker
) -> Dict[str, Any]:  # Formatted result for storage
    """
    Extract and format job result from worker response.
    
    Subclasses must implement this to format results for their job type.
    """
```

```python
@patch
def _extract_model_identifier(
    self: BaseJobManager,
    config: Dict[str, Any]  # Plugin configuration dictionary
) -> str:  # Model identifier string
    """
    Extract model identifier from plugin configuration.
    
    Override this method in subclasses to customize model identifier extraction
    based on your plugin's configuration structure.
    
    Args:
        config: Plugin configuration dictionary
        
    Returns:
        Model identifier string (e.g., model name, path, or ID)
        
    Example:
        >>> # Default behavior - tries common keys
        >>> manager._extract_model_identifier({'model_id': 'gpt-4'})
        'gpt-4'
        >>> manager._extract_model_identifier({'model': 'whisper-large'})
        'whisper-large'
        >>> manager._extract_model_identifier({'other': 'value'})
        'unknown'
    """
```

```python
@patch
async def _validate_resources(
    self: BaseJobManager,
    plugin_id: str,  # Plugin unique identifier
    plugin_config: Dict[str, Any]  # Plugin configuration
) -> Optional[str]:  # Error message if validation fails, None if successful
    """
    Validate resources before starting a job.
    
    Override this method in subclasses to implement custom resource validation logic,
    such as checking GPU availability, memory requirements, or concurrent job limits.
    
    This hook is called by start_job before creating and executing a job. If validation
    fails, start_job will raise an exception with the returned error message.
    
    Args:
        plugin_id: Unique identifier of the plugin
        plugin_config: Configuration dictionary for the plugin
        
    Returns:
        None if validation passes, or an error message string if validation fails
        
    Example:
        >>> async def _validate_resources(self, plugin_id, plugin_config):
        ...     if self.worker_process and self.current_plugin_id:
        ...         return "Another job is already running"
        ...     return None
    """
```

```python
@patch
def _on_job_completed(
    self: BaseJobManager,
    job_id: str  # ID of the completed job
) -> None
    """
    Hook called when a job completes successfully.
    
    Override this method in subclasses to implement custom post-completion logic,
    such as saving results to disk, sending notifications, or triggering follow-up tasks.
    
    This hook is called automatically by _handle_job_result after a job completes
    successfully (but not when a job fails or is cancelled).
    
    Note: This is a synchronous method called from the result monitor thread.
    If you need async operations, use asyncio.create_task() or similar.
    
    Args:
        job_id: Unique identifier of the completed job
        
    Example:
        >>> def _on_job_completed(self, job_id):
        ...     job = self.get_job(job_id)
        ...     result = self.get_job_result(job_id)
        ...     # Save to file, database, etc.
        ...     save_result_to_disk(job, result)
    """
```

```python
@patch
def _start_worker(self: BaseJobManager):
    """Start the worker process and result monitor."""
    if self.worker_process and self.worker_process.is_alive()
    "Start the worker process and result monitor."
```

```python
@patch
def _init_worker(self: BaseJobManager):
    """Send initialization message to worker with plugin configurations."""
    # Only send plugin configs if plugin registry is available
    plugin_configs = {}
    
    if self.plugin_registry
    "Send initialization message to worker with plugin configurations."
```

```python
@patch
def _restart_worker(self: BaseJobManager):
    """Restart the worker process after an error or cancellation."""
    # Track restart
    self.restart_count += 1
    self.last_restart_time = time.time()

    # Unregister old worker from resource manager if available
    if self.resource_manager and self.worker_process
    "Restart the worker process after an error or cancellation."
```

```python
@patch
def _monitor_results(self: BaseJobManager):
    """Monitor the result queue in a background thread."""
    while self.monitor_running
    "Monitor the result queue in a background thread."
```

```python
@patch
def _handle_job_result(self: BaseJobManager, result: Dict[str, Any]):
    """Handle a job result from the worker."""
    job_id = result['job_id']

    if job_id not in self.jobs
    "Handle a job result from the worker."
```

```python
@patch
def _handle_stream_chunk(self: BaseJobManager, chunk_data: Dict[str, Any]):
    """Handle a streaming chunk from the worker."""
    job_id = chunk_data.get('job_id')
    chunk = chunk_data.get('chunk', '')
    is_final = chunk_data.get('is_final', False)

    if job_id not in self._stream_buffers
    "Handle a streaming chunk from the worker."
```

```python
@patch
def _handle_worker_error(self: BaseJobManager):
    """Handle worker fatal error based on restart policy."""
    policy = self.worker_config.restart_policy

    if policy == RestartPolicy.NEVER
    "Handle worker fatal error based on restart policy."
```

```python
@patch
def get_plugin_name(
    self: BaseJobManager,
    plugin_id: str  # Plugin unique identifier
) -> Optional[str]:  # Plugin name or None
    "Get plugin name from plugin ID (requires plugin registry)."
```

```python
@patch
async def unload_plugin(
    self: BaseJobManager,
    plugin_name: str  # Name of the plugin to unload
) -> bool:  # True if successful, False otherwise
    "Unload a plugin from the worker to free resources."
```

```python
@patch
async def reload_plugin(
    self: BaseJobManager,
    plugin_name: str,  # Name of the plugin to reload
    config: Dict[str, Any]  # New configuration
) -> bool:  # True if successful, False otherwise
    "Reload a plugin with new configuration."
```

```python
@patch
async def start_job(
    self: BaseJobManager,
    plugin_id: str,  # Plugin unique identifier
    **kwargs  # Domain-specific job parameters
) -> JobType:  # Created and started job
    """
    Start a new job.
    
    Calls the _validate_resources hook before starting the job. Override that
    method in subclasses to add custom resource validation logic.
    """
```

```python
@patch
async def cancel_job(
    self: BaseJobManager,
    job_id: str  # ID of the job to cancel
) -> bool:  # True if cancellation successful
    """
    Cancel a running job by terminating the worker process.
    The worker will be automatically restarted based on restart policy.
    """
```

```python
@patch
def get_job(
    self: BaseJobManager,
    job_id: str  # Unique job identifier
) -> Optional[JobType]:  # Job object or None
    "Get a job by ID."
```

```python
@patch
def get_all_jobs(
    self: BaseJobManager
) -> List[JobType]:  # List of all jobs
    "Get all jobs."
```

```python
@patch
def get_job_result(
    self: BaseJobManager,
    job_id: str  # Unique job identifier
) -> Optional[Dict[str, Any]]:  # Job result or None
    "Get job result."
```

```python
@patch
def clear_completed_jobs(
    self: BaseJobManager
) -> int:  # Number of jobs cleared
    "Clear completed, failed, and cancelled jobs."
```

```python
@patch
async def broadcast_event(
    self: BaseJobManager,
    event_type: str,  # Event type identifier
    data: Dict[str, Any]  # Event data payload
)
    "Broadcast an event to all connected SSE clients (requires event broadcaster)."
```

```python
@patch
def check_streaming_support(
    self: BaseJobManager,
    plugin_id: str  # Plugin unique identifier
) -> bool:  # True if streaming supported
    "Check if a plugin supports streaming."
```

```python
@patch
def shutdown(self: BaseJobManager):
    """Shutdown the manager and cleanup resources."""
    # Stop result monitor
    self.monitor_running = False
    if self.result_monitor_thread
    "Shutdown the manager and cleanup resources."
```

#### Classes

```python
@dataclass
class BaseJob:
    "Base class for all job types."
    
    id: str  # Unique job identifier
    plugin_id: str  # Plugin identifier for this job
    status: str = 'pending'  # Job status: pending, running, completed, failed, cancelled
    created_at: str = field(...)  # ISO format timestamp
    started_at: Optional[str]  # When job started executing
    completed_at: Optional[str]  # When job finished
    result: Optional[Dict[str, Any]]  # Job result data
    error: Optional[str]  # Error message if failed
    metadata: Dict[str, Any] = field(...)  # Additional job metadata
    worker_pid: Optional[int]  # Process ID of worker handling this job
```

```python
class BaseJobManager:
    def __init__(
        self,
        worker_type: str,  # Type identifier (e.g., "transcription", "llm", "image-gen")
        category: Any,  # Plugin category this manager handles
        supports_streaming: bool = False,  # Whether this manager supports streaming jobs
        worker_config: Optional[WorkerConfig] = None,  # Worker configuration (uses defaults if None)
        plugin_registry: Optional[PluginRegistryProtocol] = None,  # Optional plugin registry integration
        resource_manager: Optional[ResourceManagerProtocol] = None,  # Optional resource manager integration
        event_broadcaster: Optional[EventBroadcasterProtocol] = None,  # Optional SSE event broadcaster
    )
    """
    Abstract base class for managing jobs using worker processes.
    
    Features:
    - Jobs processed sequentially in subprocess
    - Models loaded once and reused
    - True cancellation via subprocess termination
    - Automatic worker restart based on policy
    - Isolated worker process avoids duplicating web app initialization
    - Optional streaming support for incremental results
    - Optional dependency injection for plugin registry, resource manager, and event broadcaster
    """
    
    def __init__(
            self,
            worker_type: str,  # Type identifier (e.g., "transcription", "llm", "image-gen")
            category: Any,  # Plugin category this manager handles
            supports_streaming: bool = False,  # Whether this manager supports streaming jobs
            worker_config: Optional[WorkerConfig] = None,  # Worker configuration (uses defaults if None)
            plugin_registry: Optional[PluginRegistryProtocol] = None,  # Optional plugin registry integration
            resource_manager: Optional[ResourceManagerProtocol] = None,  # Optional resource manager integration
            event_broadcaster: Optional[EventBroadcasterProtocol] = None,  # Optional SSE event broadcaster
        )
        "Initialize the job manager.

All integrations (plugin_registry, resource_manager, event_broadcaster) are optional.
The manager will function without them, but features that depend on them will be disabled."
```


### config (`config.ipynb`)
> Configuration for worker processes including restart policies, timeouts, and queue sizes.

#### Import

```python
from cjm_fasthtml_workers.core.config import (
    RestartPolicy,
    WorkerConfig
)
```
#### Classes

```python
class RestartPolicy(Enum):
    "Policy for restarting worker processes after failures."
```

```python
@dataclass
class WorkerConfig:
    "Configuration for worker process behavior."
    
    request_queue_size: int = 0  # 0 = unlimited
    result_queue_size: int = 100  # Larger for streaming results
    response_queue_size: int = 10  # For synchronous command responses
    restart_policy: RestartPolicy = RestartPolicy.ON_CANCELLATION
    max_restart_attempts: int = 3
    restart_backoff_base_seconds: float = 1.0  # Base delay for exponential backoff
    restart_backoff_max_seconds: float = 60.0  # Max delay for backoff
    max_workers: int = 1  # Currently only 1 is supported
    worker_start_timeout_seconds: float = 30.0
    reload_timeout_seconds: float = 30.0
    unload_timeout_seconds: float = 10.0
    shutdown_timeout_seconds: float = 5.0
    result_monitor_poll_interval_seconds: float = 0.5
    
```


### protocol (`protocol.ipynb`)
> Protocol definitions for worker communication and plugin manager integration.

#### Import

```python
from cjm_fasthtml_workers.core.protocol import (
    WorkerRequestType,
    WorkerResponseType,
    WorkerRequest,
    WorkerResponse,
    WorkerStreamChunk,
    WorkerResult,
    PluginManagerAdapter
)
```
#### Classes

```python
class WorkerRequestType(Enum):
    "Types of requests sent to worker process."
```

```python
class WorkerResponseType(Enum):
    "Types of responses from worker process."
```

```python
@dataclass
class WorkerRequest:
    "Base structure for worker requests."
    
    type: WorkerRequestType
    data: Dict[str, Any]
    
    def to_dict(self) -> Dict[str, Any]:
            """Convert to dictionary for queue serialization."""
            return {
                'type': self.type.value,
        "Convert to dictionary for queue serialization."
    
    def from_dict(cls, data: Dict[str, Any]) -> 'WorkerRequest':
            """Create from dictionary received from queue."""
            req_type = WorkerRequestType(data['type'])
            request_data = {k: v for k, v in data.items() if k != 'type'}
        "Create from dictionary received from queue."
```

```python
@dataclass
class WorkerResponse:
    "Base structure for worker responses."
    
    type: WorkerResponseType
    data: Dict[str, Any]
    
    def to_dict(self) -> Dict[str, Any]:
            """Convert to dictionary for queue serialization."""
            return {
                'type': self.type.value,
        "Convert to dictionary for queue serialization."
    
    def from_dict(cls, data: Dict[str, Any]) -> 'WorkerResponse':
            """Create from dictionary received from queue."""
            resp_type = WorkerResponseType(data['type'])
            response_data = {k: v for k, v in data.items() if k != 'type'}
        "Create from dictionary received from queue."
```

```python
@dataclass
class WorkerStreamChunk:
    "Structure for streaming job results."
    
    job_id: str  # Unique identifier for the job
    chunk: str  # Text chunk from streaming output
    is_final: bool = False  # Whether this is the final chunk
    metadata: Optional[Dict[str, Any]]  # Optional metadata
    
    def to_dict(self) -> Dict[str, Any]:
            """Convert to dictionary for queue serialization."""
            return {
                'type': WorkerResponseType.STREAM_CHUNK.value,
        "Convert to dictionary for queue serialization."
```

```python
@dataclass
class WorkerResult:
    "Structure for job execution results."
    
    job_id: str  # Unique identifier for the job
    status: str  # 'success' or 'error'
    data: Optional[Dict[str, Any]]  # Result data on success
    error: Optional[str]  # Error message on failure
    
    def to_dict(self) -> Dict[str, Any]:
            """Convert to dictionary for queue serialization."""
            result = {
                'type': WorkerResponseType.RESULT.value,
        "Convert to dictionary for queue serialization."
```

```python
class PluginManagerAdapter(Protocol):
    """
    Protocol that plugin managers must satisfy for worker integration.
    
    Uses structural subtyping (duck typing) - plugin managers don't need to
    explicitly inherit from this, they just need to implement these methods.
    """
    
    def discover_plugins(self) -> list:
            """
            Discover available plugins.
    
            Returns:
                List of plugin metadata/data objects
            """
            ...
    
        def load_plugin(self, plugin_data: Any, config: Dict[str, Any]) -> None
        "Discover available plugins.

Returns:
    List of plugin metadata/data objects"
    
    def load_plugin(self, plugin_data: Any, config: Dict[str, Any]) -> None:
            """
            Load a plugin with configuration.
    
            Args:
                plugin_data: Plugin metadata/data from discovery
                config: Plugin configuration dictionary
            """
            ...
    
        def execute_plugin(self, plugin_name: str, **params) -> Any
        "Load a plugin with configuration.

Args:
    plugin_data: Plugin metadata/data from discovery
    config: Plugin configuration dictionary"
    
    def execute_plugin(self, plugin_name: str, **params) -> Any:
            """
            Execute a plugin with given parameters.
    
            Args:
                plugin_name: Name of the plugin to execute
                **params: Plugin-specific parameters
    
            Returns:
                Plugin execution result
            """
            ...
    
        def execute_plugin_stream(self, plugin_name: str, **params) -> Iterator[str]
        "Execute a plugin with given parameters.

Args:
    plugin_name: Name of the plugin to execute
    **params: Plugin-specific parameters

Returns:
    Plugin execution result"
    
    def execute_plugin_stream(self, plugin_name: str, **params) -> Iterator[str]:
            """
            Execute a plugin with streaming output.
    
            Args:
                plugin_name: Name of the plugin to execute
                **params: Plugin-specific parameters
    
            Yields:
                String chunks from plugin execution
            """
            ...
    
        def reload_plugin(self, plugin_name: str, config: Optional[Dict[str, Any]] = None) -> None
        "Execute a plugin with streaming output.

Args:
    plugin_name: Name of the plugin to execute
    **params: Plugin-specific parameters

Yields:
    String chunks from plugin execution"
    
    def reload_plugin(self, plugin_name: str, config: Optional[Dict[str, Any]] = None) -> None:
            """
            Reload a plugin with new configuration.
    
            Args:
                plugin_name: Name of the plugin to reload
                config: New configuration (None to unload)
            """
            ...
    
        def unload_plugin(self, plugin_name: str) -> None
        "Reload a plugin with new configuration.

Args:
    plugin_name: Name of the plugin to reload
    config: New configuration (None to unload)"
    
    def unload_plugin(self, plugin_name: str) -> None:
            """
            Unload a plugin to free resources.
    
            Args:
                plugin_name: Name of the plugin to unload
            """
            ...
    
        def check_streaming_support(self, plugin_name: str) -> bool
        "Unload a plugin to free resources.

Args:
    plugin_name: Name of the plugin to unload"
    
    def check_streaming_support(self, plugin_name: str) -> bool
        "Check if a plugin supports streaming execution.

Args:
    plugin_name: Name of the plugin to check

Returns:
    True if plugin supports streaming"
```


### protocols (`protocols.ipynb`)
> Optional integration protocols for plugin registries, resource management, and event broadcasting.

#### Import

```python
from cjm_fasthtml_workers.extensions.protocols import (
    PluginRegistryProtocol,
    ResourceManagerProtocol,
    EventBroadcasterProtocol
)
```
#### Classes

```python
class PluginRegistryProtocol(Protocol):
    "Protocol for plugin registry integration."
    
    def get_plugins_by_category(self, category: Any) -> list:
            """
            Get all plugins in a specific category.
            
            Args:
                category: Plugin category (can be enum, string, etc.)
                
            Returns:
                List of plugin metadata objects
            """
            ...
        
        def get_plugin(self, plugin_id: str) -> Any
        "Get all plugins in a specific category.

Args:
    category: Plugin category (can be enum, string, etc.)
    
Returns:
    List of plugin metadata objects"
    
    def get_plugin(self, plugin_id: str) -> Any:
            """
            Get a specific plugin by ID.
            
            Args:
                plugin_id: Unique plugin identifier
                
            Returns:
                Plugin metadata object or None
            """
            ...
        
        def load_plugin_config(self, plugin_id: str) -> Dict[str, Any]
        "Get a specific plugin by ID.

Args:
    plugin_id: Unique plugin identifier
    
Returns:
    Plugin metadata object or None"
    
    def load_plugin_config(self, plugin_id: str) -> Dict[str, Any]
        "Load configuration for a plugin.

Args:
    plugin_id: Unique plugin identifier
    
Returns:
    Plugin configuration dictionary"
```

```python
class ResourceManagerProtocol(Protocol):
    "Protocol for resource management integration."
    
    def register_worker(
            self,
            pid: int,  # Worker process ID
            worker_type: str  # Type of worker (e.g., 'transcription', 'llm')
        ) -> None
        "Register a new worker process.

Args:
    pid: Process ID of the worker
    worker_type: Type identifier for the worker"
    
    def unregister_worker(self, pid: int) -> None:
            """
            Unregister a worker process.
            
            Args:
                pid: Process ID of the worker to unregister
            """
            ...
        
        def update_worker_state(
            self,
            pid: int,  # Worker process ID
            status: Optional[str] = None,  # Worker status: 'idle', 'running', etc.
            job_id: Optional[str] = None,  # Current job ID (None if idle)
            plugin_name: Optional[str] = None,  # Currently loaded plugin name
            plugin_id: Optional[str] = None,  # Currently loaded plugin ID
            loaded_model: Optional[str] = None,  # Currently loaded model identifier
            config: Optional[Dict[str, Any]] = None,  # Current plugin configuration
        ) -> None
        "Unregister a worker process.

Args:
    pid: Process ID of the worker to unregister"
    
    def update_worker_state(
            self,
            pid: int,  # Worker process ID
            status: Optional[str] = None,  # Worker status: 'idle', 'running', etc.
            job_id: Optional[str] = None,  # Current job ID (None if idle)
            plugin_name: Optional[str] = None,  # Currently loaded plugin name
            plugin_id: Optional[str] = None,  # Currently loaded plugin ID
            loaded_model: Optional[str] = None,  # Currently loaded model identifier
            config: Optional[Dict[str, Any]] = None,  # Current plugin configuration
        ) -> None
        "Update worker state information.

This method uses optional parameters to allow partial updates.
Only provided parameters will be updated; None values can be used
to clear state (e.g., when unloading a plugin).

Args:
    pid: Process ID of the worker
    status: Worker status (e.g., 'idle', 'running')
    job_id: Current job identifier (None if no job running)
    plugin_name: Name of currently loaded plugin
    plugin_id: Unique identifier of currently loaded plugin
    loaded_model: Identifier of currently loaded model
    config: Current plugin configuration dictionary"
```

```python
class EventBroadcasterProtocol(Protocol):
    "Protocol for SSE event broadcasting."
    
    async def broadcast(
            self,
            event_type: str,  # Event type identifier
            data: Dict[str, Any]  # Event data payload
        ) -> None
        "Broadcast an event to all connected clients.

Args:
    event_type: Type of event (e.g., 'job:started', 'job:completed')
    data: Event data to broadcast"
```


### worker (`worker.ipynb`)
> Generic worker process for executing plugin-based jobs in isolated subprocesses.

#### Import

```python
from cjm_fasthtml_workers.core.worker import (
    base_worker_process
)
```

#### Functions

```python
def base_worker_process(
    request_queue: multiprocessing.Queue,  # Queue for receiving job requests from parent
    result_queue: multiprocessing.Queue,  # Queue for sending job results back to parent
    response_queue: multiprocessing.Queue,  # Queue for sending command responses back to parent
    plugin_manager_factory: Callable[[], PluginManagerAdapter],  # Factory function that creates a plugin manager instance
    result_adapter: Optional[Callable[[Any], Dict[str, Any]]] = None,  # Optional function to adapt plugin results to dict format
    supports_streaming: bool = False  # Whether this worker supports streaming execution
)
    """
    Generic long-lived worker process that handles job execution.
    
    This process:
    1. Receives plugin configurations through init message
    2. Initializes the plugin manager once with provided configs
    3. Loads models lazily on first use (and keeps them loaded)
    4. Processes jobs from the request queue
    5. Sends job results back via result queue
    6. Sends command responses back via response queue
    7. Can be terminated for cancellation
    
    Protocol:
        - Parent sends 'init' message first with plugin configurations
        - Then sends 'execute' messages for jobs
        - Sends 'unload' to unload a specific plugin
        - Sends 'reload' to reload a plugin with new configuration
        - Sends 'get_state' to query current worker state
        - Sends 'stop' to gracefully shutdown
    """
```
